In [1]:
import pandas as pd

In [2]:
# Create dataframe from CSV file
df = pd.read_csv('sheets/organizations.csv', na_filter=False)

In [3]:
# Rename columns to map to Zendesk fields
df = df.rename(columns={'id': 'external_id'})

In [4]:
# Ensure required columns are unique
print('{:<30}{}'.format("\'external_id\' is unique:", df['external_id'].is_unique))
print('{:<30}{}'.format("\'name\' is unique:", df['name'].is_unique))

'external_id' is unique:      True
'name' is unique:             True


In [5]:
# Assign values to new 'organization_fields' column
new_col = [{'merchant_id': df['merchant_id'][i]} for i in range(len(df))]
# Add new column to dataframe
df['organization_fields'] = new_col

In [6]:
# Remove unnecessary 'merchant_id' column
del df['merchant_id']

In [7]:
# Convert 'external_id' column to string to match required input
df['external_id'] = df['external_id'].apply(str)

In [8]:
# Transform 'domain_names' and 'tags' into a list of strings
def str_to_list(row, col) -> list:
    if isinstance(row[col], str):
        if len(row[col]) < 3:
            return ['mcopland'] if col == 'tags' else []
        else:
            # Split string into list of strings
            new_list = row[col][1:-1].split(', ')
            # Remove unnecessary characters
            for i, entry in enumerate(new_list):
                if entry[1] == '(' and entry[-2] == ')':
                    # Strip single quotes and parentheses
                    new_list[i] = entry[2:-2]
                else:
                    # Strip single quotes only
                    new_list[i] = entry[1:-1]
            if col == 'tags':
                # Tag all organizations with my name so that they can be easily located later
                new_list.append('mcopland')
            return new_list
    # Default return
    return row[col]
            
df['domain_names'] = df.apply(lambda row: str_to_list(row, 'domain_names'), axis=1)
df['tags'] = df.apply(lambda row: str_to_list(row, 'tags'), axis=1)

In [9]:
# View dataframe information
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   external_id          125 non-null    object
 1   name                 125 non-null    object
 2   domain_names         125 non-null    object
 3   details              125 non-null    object
 4   notes                125 non-null    object
 5   tags                 125 non-null    object
 6   organization_fields  125 non-null    object
dtypes: object(7)
memory usage: 7.0+ KB


,external_id,name,domain_names,details,notes,tags,organization_fields
0,15976972608,"1,000 Bulbs","[1000bulbs.com, protonmail.com]",,,[mcopland],{'merchant_id': 'apac'}
1,15976972628,1-800-Flowers.Com Inc,[],,Hello\nthis is a new line\nand another,[mcopland],{'merchant_id': 'amer'}
2,15976972648,1-Stop Connections,[1-stop.biz],,,[mcopland],{'merchant_id': 'latam'}
3,3535325388,1077207-Moneygram International,[],,,[mcopland],{'merchant_id': 'emea'}
4,360041750613,123456789,[],,,[mcopland],{'merchant_id': 'amer'}


In [10]:
# Split df into chunks of 100 for batch import
n = 100
chunks = [df[i:i+n] for i in range(0, df.shape[0], n)]
print(len(chunks))

2


In [11]:
# Export to JSON file
for i in range(len(chunks)):
    filename = f'./organizations_{i}.json'
    # records : list [{column -> value}, ... , {column -> value}]
    chunks[i].to_json(filename, orient='records')